In [ ]:
pip install openai

In [ ]:
!pip install -q openai

import traceback
import re
from openai import OpenAI
import os

GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

# Replace with your Groq API key
client = OpenAI(
    api_key=GROQ_API_KEY,
    base_url="https://api.groq.com/openai/v1"
)

def clean_code_block(code):
    if "```" in code:
        match = re.search(r"```(?:python)?\s*([\s\S]+?)```", code)
        if match:
            return match.group(1).strip()
    return code.strip()

def generate_code(task, history=""):
    prompt = f"""
You are a Python coding assistant.
Write a complete and runnable Python function or script for the following task.

TASK:
{task}

PREVIOUS ATTEMPT (if any):
{history}

Only return the Python code inside a single Python code block. Do not include explanations.
"""
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": "You are a helpful Python coding agent."},
            {"role": "user", "content": prompt}
        ]
    )
    code_raw = response.choices[0].message.content
    return clean_code_block(code_raw)

def run_code(code):
    try:
        local_vars = {}
        exec(code, {}, local_vars)
        return "Code ran successfully!", None
    except Exception:
        return None, traceback.format_exc()

def self_improving_agent():
    task = input("Enter the coding task: ")
    max_attempts = 5
    attempt = 0
    history = ""

    while attempt < max_attempts:
        print(f"\n🧠 Attempt {attempt + 1}")
        code = generate_code(task, history)
        print("\n📝 Generated Code:\n")
        print(code)

        result, error = run_code(code)
        if result:
            print("\n✅ Success:", result)
            break
        else:
            print("\n❌ Error:\n", error)
            history = f"The following code failed:\n{code}\nWith error:\n{error}"
            attempt += 1
    else:
        print("\n❗ Max attempts reached. Agent couldn't fix the issue.")

# 🔁 Run the agent
self_improving_agent()


Enter the coding task: sum of 2 numbers

🧠 Attempt 1

📝 Generated Code:

def sum_of_numbers(a, b):
    return a + b

✅ Success: Code ran successfully!
